In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functools as ft
import multiprocessing as mp
import sys

In [3]:
%matplotlib inline

In [4]:
from hgflask import app
import hgflask
from flask import jsonify, request

In [5]:
import hgtiles

processes = {}

### HiGlass Server

In [6]:
hgflask.TILESETS = [{  
    'uuid': "abc",
    'filetype': "bigwig",
    'datatype': "vector",
    'url': 'http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878Ctcfsc15914c20StdSig.bigWig',
    'private': False,
    'name': "blabla",
    'coordSystem': "hg19",
    'coordSystem2': "hg19",
    'created': "2017-02-02T21:10:56.265085Z",
    },
    {
        'uuid': 'a',
        "name": "my cooler",
        'filetype': 'cooler',
        'datatype': 'matrix',
        'filepath': '/Users/pete/Dropbox/paper-data/phillippy-data/out.mcool',
    }
]

import slugid

for puid in processes:
    processes[puid].terminate()

processes = {}
print(hgflask.TILESETS)
# we're going to assign a uuid to each server process so that if anything
# goes wrong, the variable referencing the process doesn't get lost
uuid = slugid.nice()
processes[uuid] = mp.Process(target=ft.partial(app.run, debug=True, port=4007, host='0.0.0.0', use_reloader=False))
processes[uuid].start()

[{'uuid': 'abc', 'filetype': 'bigwig', 'datatype': 'vector', 'url': 'http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878Ctcfsc15914c20StdSig.bigWig', 'private': False, 'name': 'blabla', 'coordSystem': 'hg19', 'coordSystem2': 'hg19', 'created': '2017-02-02T21:10:56.265085Z'}, {'uuid': 'a', 'name': 'my cooler', 'filetype': 'cooler', 'datatype': 'matrix', 'filepath': '/Users/pete/Dropbox/paper-data/phillippy-data/out.mcool'}]


 * Running on http://0.0.0.0:4007/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Feb/2018 11:55:34] "GET /api/v1/chrom-sizes/?id=a HTTP/1.1" 404 -
127.0.0.1 - - [04/Feb/2018 11:55:34] "OPTIONS /api/v1/tileset_info/?d=a&s=crV3fu54RLy3g6W4b_EdwA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:56:09] "GET /api/v1/tiles/?d=a.0.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:56:34] "GET /api/v1/chrom-sizes/?id=a HTTP/1.1" 404 -
127.0.0.1 - - [04/Feb/2018 11:56:34] "OPTIONS /api/v1/tileset_info/?d=a&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:56:34] "GET /api/v1/chrom-sizes/?id=a HTTP/1.1" 404 -
127.0.0.1 - - [04/Feb/2018 11:56:34] "GET /api/v1/tileset_info/?d=a&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:56:34] "OPTIONS /api/v1/tiles/?d=a.1.0.0&d=a.1.0.1&d=a.1.1.1&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:56:35] "GET /api/v1/tiles/?d=a.1.0.0&d=a.1.0.1&d=a.1.1.1&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - 

127.0.0.1 - - [04/Feb/2018 11:59:35] "GET /api/v1/tiles/?d=a.7.75.76&d=a.7.75.77&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:59:41] "OPTIONS /api/v1/tiles/?d=a.6.37.37&d=a.6.37.38&d=a.6.38.38&d=a.6.37.39&d=a.6.38.39&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:59:41] "GET /api/v1/tiles/?d=a.6.37.37&d=a.6.37.38&d=a.6.38.38&d=a.6.37.39&d=a.6.38.39&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:59:42] "OPTIONS /api/v1/tiles/?d=a.5.18.18&d=a.5.18.19&d=a.5.19.19&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:59:42] "GET /api/v1/tiles/?d=a.5.18.18&d=a.5.18.19&d=a.5.19.19&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:59:44] "OPTIONS /api/v1/tiles/?d=a.5.19.20&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:59:44] "GET /api/v1/tiles/?d=a.5.19.20&s=EtZZWE6LQSu9c15bpp9bfA HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2018 11:59:46] "OPTIONS /api/v1/tiles/?d

In [7]:
!curl localhost:4007/api/v1/tiles/?d=a.0.0.0

{
  "a.0.0.0": {


    "dense": "...

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [8]:
!curl higlass.io/api/v1/tiles/?d=CQMd6V_cRw6iCI_-Unl3PQ.0.0.0

{"CQMd6V_cRw6iCI_-Unl3PQ.0.0.0": {"min_value": 0.0, "max_value": 1.2987070083618164, "dense": "...

...

...", "dtype": "float16"}}

In [21]:
a = np.random.random((4,3))
print(a)

[[ 0.04855233  0.75723857  0.08055191]
 [ 0.1713757   0.58781498  0.01928301]
 [ 0.64798268  0.37728143  0.954238  ]
 [ 0.55084939  0.95120137  0.56308623]]


In [23]:
a.T.reshape((a.shape[1],-1,2)).reduce()

AttributeError: 'numpy.ndarray' object has no attribute 'reduce'

In [25]:
a = np.array([1,2,3,4])
a.reshape((-1,2)).sum(axis=1)

array([3, 7])

In [26]:
a.reshape((-1,2)).max(axis=1)

array([2, 4])

In [27]:
b = np.array([[1,2],[3,4],[5,6],[3,3]])

In [29]:
b = [[1,2],[3,4],[5,6],[3,3]]
c = []
for i in range(0,len(b) - 1,2):
    if sum(b[i]) > sum(b[i+1]):
        c += [b[i]]
    else:
        c += [b[i+1]]
print(c)

[[3, 4], [5, 6]]
